In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from utility import get_output_path

plt.style.use('ggplot')
pd.set_option('display.float_format', '{:.2f}'.format)

compare_items = [
    'end_balance',
    'total_days',
    'profit_days',
    'total_trade_count',
    'max_ddpercent',
#     'max_drawdown_duration',
    'total_return',
    'annual_return',
    'return_std',
    'daily_return',
    'sharpe_ratio',
    'return_drawdown_ratio'
]
test_folder = Path(r'D:\research_tools\backtest\result\rb_1h_2014010120191201_turtle_default')
# test_folder = Path(r'E:\project\research_tools\backtest\result\rb_1h_2014010120191201_turtle_default')

In [ ]:
# 成交结果对比分析
cnt_items = [item + '_cnt' for item in compare_items]
seg_items = [item + '_seg' for item in compare_items]
keep_item = cnt_items + seg_items
keep_item.insert(0, 'name')

fp = get_output_path('batch_result.csv')
df = pd.read_csv(fp, index_col=0)
df = df[keep_item]

In [ ]:
for name in compare_items:
    df[name] = df[name + '_cnt'] - df[name + '_seg']
keep_items = compare_items[:]
keep_items.insert(0, 'name')
df = df[keep_items]
df.describe()

In [ ]:
%matplotlib notebook
df.return_std.hist(bins=50)

In [ ]:
trade_kwargs = {'index_col': 0, 'parse_dates': True}
pnl_kwargs = {'index_col': 0, 'parse_dates': True, 'usecols': [0, 1, 2, 4, 5, 6, 10, 11, 12, 13]}
result_kwargs = {'index_col': 0}
trade_cont = pd.read_csv(test_folder.joinpath('trade_continuous.csv'), **trade_kwargs)
trade_seg = pd.read_csv(test_folder.joinpath('trade_seg.csv'), **trade_kwargs)
pnl_cont = pd.read_csv(test_folder.joinpath('pnl_continuous.csv'), **pnl_kwargs)
pnl_seg = pd.read_csv(test_folder.joinpath('pnl_seg.csv'), **pnl_kwargs)
res_cont = pd.read_csv(test_folder.joinpath('result_continuous.csv'), **result_kwargs)
res_seg = pd.read_csv(test_folder.joinpath('result_seg.csv'), **result_kwargs)
res_cont = res_cont[compare_items]
res_seg = res_seg[compare_items]

In [ ]:
test = res_cont.copy()
s = test.iloc[0].copy()
s['hey'] = 22
s

In [ ]:
df = pd.read_csv(test_folder.joinpath('source_res_seg.csv'), **result_kwargs)
idx = df['drawdown'].idxmin()
df['balance'][:idx].argmax()


In [ ]:
trade_cont.head()
# trade_cont.info()
pnl_cont.head()

In [ ]:
trade_seg.head()
pnl_seg.head()

In [ ]:
trade_comp = trade_cont.merge(trade_seg, left_index=True, right_index=True, how='outer', suffixes=('_cont','_seg'))
pnl_comp = pnl_cont.merge(pnl_seg, left_index=True, right_index=True, how='outer', suffixes=('_cont','_seg'))
res_comp = res_cont.merge(res_seg, left_index=True, right_index=True, how='outer', suffixes=('_cont','_seg'))

In [ ]:
# trade_comp.head()
# 选取拼接合约成交点位和分段回测不同的单
df = trade_comp[trade_comp.isnull().T.any()]
df
# num = len(df[df['direction_cont'].notnull()])
# num / len(trade_cont)

In [ ]:
df = pnl_comp[pnl_comp.isnull().T.any()]
df

In [ ]:
# 每日盈亏数据不一致的记录
pnl_comp.head()
df = pnl_comp[pnl_comp.net_pnl_cont != pnl_comp.net_pnl_seg]
df
# len(df) / len(pnl_seg)

In [ ]:
# 选取成交点位相同但是方向不一致的单
df = trade_comp[~trade_comp.isnull().T.any()]
df[df.direction_cont != df.direction_seg]

In [ ]:
trade_comp.to_csv('result/rb_trade_comp.csv')